In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import scipy as scipy

In [2]:
from scipy.stats import poisson

In [3]:
from scipy import optimize

In [4]:
sf = pd.read_csv('../data/raw_data/san_francisco.csv', low_memory=False)

In [5]:
#sj = pd.read_csv('../data/raw_data/san_jose.csv', low_memory=False)

In [6]:
sf

,raw_row_number,date,time,location,lat,lng,district,subject_age,subject_race,subject_sex,...,citation_issued,warning_issued,outcome,contraband_found,search_conducted,search_vehicle,search_basis,reason_for_stop,raw_search_vehicle_description,raw_result_of_contact_description
0,869921,2014-08-01,00:01:00,MASONIC AV & FELL ST,37.773004,-122.445873,NaN,NaN,asian/pacific islander,female,...,False,True,warning,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Warning
1,869922,2014-08-01,00:01:00,GEARY&10TH AV,37.780898,-122.468586,NaN,NaN,black,male,...,True,False,citation,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Citation
2,869923,2014-08-01,00:15:00,SUTTER N OCTAVIA ST,37.786919,-122.426718,NaN,NaN,hispanic,male,...,True,False,citation,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Citation
3,869924,2014-08-01,00:18:00,3RD ST & DAVIDSON,37.746380,-122.392005,NaN,NaN,hispanic,male,...,False,True,warning,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Warning
4,869925,2014-08-01,00:19:00,DIVISADERO ST. & BUSH ST.,37.786348,-122.440003,NaN,NaN,white,male,...,True,False,citation,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Citation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905065,893005,2014-10-31,23:47:00,ORTEGA ST & 38TH AVE,37.751482,-122.496725,NaN,NaN,white,male,...,False,True,warning,NaN,False,False,NaN,Moving Violation,No Search,Warning
905066,893006,2014-10-31,23:48:00,DUBOCE/BELCHER,37.769388,-122.430272,NaN,NaN,asian/pacific islander,male,...,True,False,citation,NaN,False,False,NaN,Moving Violation,No Search,Citation
905067,893007,2014-10-31,23:49:00,NEW MONTGOMERY & STEVENSON,37.788297,-122.401527,NaN,NaN,white,male,...,True,False,citation,NaN,False,False,NaN,Moving Violation,No Search,Citation
905068,893008,2014-10-31,23:50:00,BURKE & 3RD,37.745006,-122.387067,NaN,NaN,black,male,...,False,True,warning,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Warning


In [7]:
#sj

In [8]:
stop_by_race = sf["subject_race"].value_counts()

In [9]:
stop_by_race

white                     372318
asian/pacific islander    157684
black                     152196
hispanic                  116014
other                     106858
Name: subject_race, dtype: int64

In [10]:
race_index = ['white', 'asian/pacific islander', 'black', 'hispanic', 'other']
# http://www.bayareacensus.ca.gov/bayarea.htm

In [11]:
sf_pop_list = [.402, .36+.005, .056, .152, .007+.045
              ] # https://www.census.gov/quickfacts/sanfranciscocountycalifornia

In [12]:
ba_pop_list = [.424, 0.230+0.003, 0.064, 0.235, 0.038 ]

In [13]:
sf_pop = pd.Series(sf_pop_list, index = race_index)

In [14]:
ba_pop = pd.Series(ba_pop_list, index = race_index)

In [15]:
sf_pop

white                     0.402
asian/pacific islander    0.365
black                     0.056
hispanic                  0.152
other                     0.052
dtype: float64

In [16]:
pop_by_race = round(881549*sf_pop)

In [17]:
pop_by_race

white                     354383.0
asian/pacific islander    321765.0
black                      49367.0
hispanic                  133995.0
other                      45841.0
dtype: float64

In [18]:
ba_pop_by_race = round(7150739 * ba_pop)

In [19]:
ba_pop_by_race

white                     3031913.0
asian/pacific islander    1666122.0
black                      457647.0
hispanic                  1680424.0
other                      271728.0
dtype: float64

In [20]:
def bic(params, data, minus_max_like):
    result = len(params)*np.log(data.count()) + 2*minus_max_like
    return result

In [21]:
def ln_like_1(mu, stops, pops):
    lnl=0
    for idx in stop_by_race.index:
        expect = mu*pops[idx]
        pois = scipy.stats.poisson.pmf(stops[idx], expect)
        lnl = lnl + np.log(pois)
        print("{}: expect {}, got {}. Prob: {}".format(idx, expect, stops[idx], pois))
        
    return lnl

In [22]:
def ln_like_2(mu, stops, pops, transient):
    lnl=0
    for idx in stop_by_race.index:
        expect = mu*(pops[idx] + transient[idx])
        pois = scipy.stats.poisson.pmf(stops[idx], expect)
        if(pois <= 0):
            pois = 1e-180
        lnl = lnl + np.log(pois)
        #print("{}: expect {}, got {}. Prob: {}".format(idx, expect, stops[idx], pois))
        
    return lnl

In [23]:
(stop_by_race/pop_by_race).values

array([1.05060909, 0.49005952, 3.08295015, 0.86580843, 2.33105735])

In [25]:
def ln_like_opt(x0, stop_by_race, pop_by_race, ba_pop_by_race):
    return -ln_like_2(x0[0], stop_by_race, pop_by_race, x0[1:] * ba_pop_by_race)

In [26]:
initial = [0.48, 0.14, 0, 0.58, 0.06, 0.7]

In [27]:
ln_like_opt(initial, stop_by_race, pop_by_race, ba_pop_by_race)

310.1328243294993

In [28]:
res = optimize.minimize(ln_like_opt, args=(stop_by_race, pop_by_race, ba_pop_by_race), 
                       x0=initial, method = 'Nelder-Mead', options={'maxiter':500})

In [29]:
res

 final_simplex: (array([[0.48702276, 0.1352592 , 0.00120557, 0.57498163, 0.06202079,
        0.63875725],
       [0.48700965, 0.13526957, 0.00120778, 0.57498887, 0.06201964,
        0.63878369],
       [0.48701569, 0.13526547, 0.00120472, 0.57500042, 0.0620189 ,
        0.63876835],
       [0.48701628, 0.13526054, 0.00120437, 0.57500253, 0.0620191 ,
        0.63876724],
       [0.48701947, 0.13526262, 0.00120422, 0.57499602, 0.06202133,
        0.63878215],
       [0.4870232 , 0.13525883, 0.00120564, 0.57499102, 0.06201688,
        0.63878661],
       [0.48702953, 0.13525826, 0.00120598, 0.57495466, 0.06201619,
        0.63877224]]), array([34.57948905, 34.57949662, 34.57951365, 34.57952611, 34.57953688,
       34.57954331, 34.57956223]))
           fun: 34.57948905394005
       message: 'Optimization terminated successfully.'
          nfev: 248
           nit: 150
        status: 0
       success: True
             x: array([0.48702276, 0.1352592 , 0.00120557, 0.57498163, 0.06202079,

In [30]:
minus_max_like_2 = ln_like_opt(res.x, stop_by_race, pop_by_race, ba_pop_by_race)

In [31]:
bic(initial, stop_by_race, minus_max_like_2)

78.81560558248471

In [32]:
def ln_like_3(mu, stops, pops, transient):
    lnl=0
    for idx in stop_by_race.index:
        expect = mu[idx]*(pops[idx] + transient[idx])
        pois = scipy.stats.poisson.pmf(stops[idx], expect)
        if(pois <= 0):
            pois = 1e-180
        lnl = lnl + np.log(pois)
        #print("{}: expect {}, got {}. Prob: {}".format(idx, expect, stops[idx], pois))
        
    return lnl

In [33]:
def ln_like_opt_3(x0, stop_by_race, pop_by_race, ba_pop_by_race):
    mu = pd.Series(x0[0:5], index = race_index)
    return -ln_like_3(mu, stop_by_race, pop_by_race, x0[5:] * ba_pop_by_race)

In [34]:
race_index

['white', 'asian/pacific islander', 'black', 'hispanic', 'other']

In [35]:
initial_3 = (stop_by_race/pop_by_race).values

In [36]:
initial_3 = np.append(initial_3, [0, 0, 0, 0, 0])

In [37]:
initial_3

array([1.05060909, 0.49005952, 3.08295015, 0.86580843, 2.33105735,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [38]:
ln_like_opt_3(initial_3, stop_by_race, pop_by_race, ba_pop)

34.57944478497666

In [39]:
res = optimize.minimize(ln_like_opt_3, args=(stop_by_race, pop_by_race, ba_pop_by_race), 
                       x0=initial_3, method = 'Nelder-Mead', options={'maxiter':500})

In [40]:
res

 final_simplex: (array([[1.05053872e+00, 4.90037165e-01, 3.08265317e+00, 8.65717390e-01,
        2.33088215e+00, 7.82782795e-06, 8.81393923e-06, 1.03929019e-05,
        8.38435166e-06, 1.26845293e-05],
       [1.05053231e+00, 4.90035134e-01, 3.08262609e+00, 8.65709093e-01,
        2.33086618e+00, 8.54225081e-06, 9.61431542e-06, 1.13418217e-05,
        9.14850812e-06, 1.38362220e-05],
       [1.05053555e+00, 4.90036148e-01, 3.08263976e+00, 8.65713270e-01,
        2.33087422e+00, 8.18333460e-06, 9.21438759e-06, 1.08652695e-05,
        8.76456481e-06, 1.32561109e-05],
       [1.05053083e+00, 4.90034660e-01, 3.08261989e+00, 8.65707191e-01,
        2.33086250e+00, 8.70569778e-06, 9.79882075e-06, 1.15573920e-05,
        9.32576828e-06, 1.41027069e-05],
       [1.05054009e+00, 4.90037596e-01, 3.08265892e+00, 8.65719138e-01,
        2.33088544e+00, 7.67594261e-06, 8.64437364e-06, 1.01927390e-05,
        8.22305060e-06, 1.24411244e-05],
       [1.05053272e+00, 4.90035253e-01, 3.08262781e+00, 8.

In [41]:
minus_max_like_3 = ln_like_opt_3(res.x, stop_by_race, pop_by_race, ba_pop)

In [42]:
bic(initial_3, stop_by_race, minus_max_like_3)

85.25856560948182

In [43]:
def ln_like_opt_4(x0, stop_by_race, pop_by_race, ba_pop_by_race):
    mu = pd.Series(x0[0:5], index = race_index)
    return -ln_like_3(mu, stop_by_race, pop_by_race, x0[5] * ba_pop_by_race)

In [44]:
initial_4 = np.append((stop_by_race/pop_by_race).values, 0.05)

In [45]:
res = optimize.minimize(ln_like_opt_4, args=(stop_by_race, pop_by_race, ba_pop_by_race), 
                       x0=initial_4, method = 'Nelder-Mead', options={'maxiter':500})

In [46]:
res

 final_simplex: (array([[1.05060909, 0.49005952, 3.08295015, 0.86580843, 2.33105735,
        0.05      ],
       [1.05063474, 0.49005952, 3.08295015, 0.86580843, 2.33105735,
        0.05      ],
       [1.05060909, 0.49007148, 3.08295015, 0.86580843, 2.33105735,
        0.05      ],
       [1.05060909, 0.49005952, 3.08302542, 0.86580843, 2.33105735,
        0.05      ],
       [1.05060909, 0.49005952, 3.08295015, 0.86582956, 2.33105735,
        0.05      ],
       [1.05060909, 0.49005952, 3.08295015, 0.86580843, 2.33111426,
        0.05      ],
       [1.05060909, 0.49005952, 3.08295015, 0.86580843, 2.33105735,
        0.05000122]]), array([2072.32658369, 2072.32658369, 2072.32658369, 2072.32658369,
       2072.32658369, 2072.32658369, 2072.32658369]))
           fun: 2072.326583694641
       message: 'Optimization terminated successfully.'
          nfev: 95
           nit: 12
        status: 0
       success: True
             x: array([1.05060909, 0.49005952, 3.08295015, 0.86580843,

In [47]:
ln_like_opt_4(res.x, stop_by_race, pop_by_race, ba_pop)

34.57944478651916

In [48]:
stop_by_race.count()

5

In [49]:
bic(res.x, stop_by_race, ln_like_opt_4(res.x, stop_by_race, pop_by_race, ba_pop))

78.81551704764293

In [63]:
sf[sf["subject_race"]=="black"]["reason_for_stop"].value_counts()

Moving Violation                                              75450
Mechanical or Non-Moving Violation (V.C.)                     73479
MPC Violation                                                   930
BOLO/APB/Warrant                                                646
Assistance to Motorist                                          240
DUI Check                                                       188
Traffic Collision                                               169
Moving Violation|Mechanical or Non-Moving Violation (V.C.)       29
Moving Violation|MPC Violation                                    2
Moving Violation|BOLO/APB/Warrant                                 1
Name: reason_for_stop, dtype: int64

In [64]:
sf[sf["subject_race"]=="white"]["reason_for_stop"].value_counts()

Moving Violation                                                                                        244182
Mechanical or Non-Moving Violation (V.C.)                                                               125184
MPC Violation                                                                                             1004
Traffic Collision                                                                                          475
DUI Check                                                                                                  395
Assistance to Motorist                                                                                     329
BOLO/APB/Warrant                                                                                           208
Moving Violation|Mechanical or Non-Moving Violation (V.C.)                                                  74
Moving Violation|MPC Violation                                                                               4
M